# SHAP
***
## TreeExplainer para explicar clasificación de eventos
* Datos obtenidos de https://www.openml.org/d/23512
* Cada evento es representado por un conjunto de 28 variables, donde 21 variables son de bajo nivel correspondientes a propiedades físicas medidas por el detector, y 7 variables de alto nivel, provenientes d las anteriores.
* Algunas variables son:
|Type| Variable  | Description   |
|---| --- | --- |
|low-level|lepton pT |  Momentum of the lepton|
|low-level|lepton eta | Pseudorapidity eta of the lepton|
|low-level|lepton phi | Azimuthal angle phi of the lepton|
|low-level|Missing energy magnitude | Energy not detected|
|| ... | ...|
|high-level|m_jlv| Mass jet ($j$), lepton ($l$, electrons or muons), neutrino $\nu$| 
|high-level|m_bb| Mass quarks $b$|	
|high-level|m_wbb| Mass boson $W$ and quarks $b$|
|high-level|m_wwbb|Mass bosons $W$ and quarks $b$|



- Más detalles en [Baldi et al] Baldi, P., Sadowski, P., & Whiteson, D. (2014). Searching for exotic particles in high-energy physics with deep learning. Nature communications, 5(1), 1-9 [(link)](https://www.nature.com/articles/ncomms5308).

<img src="img/signal_back.jpg" width="300">
Imagen obtenida de Baldi et al.

- **Problema:** Clasificación binario de eventos HEP, para identificar la señal del background
- señal: $gg \rightarrow H^0 \rightarrow W^{\mp} H^{\pm} \rightarrow W^{\mp} W^{\pm} h^0 \rightarrow W^{\mp} W^{\pm} b \bar{b}$. 
 This signal process is the fusion of two gluons into a heavy electrically neutral Higgs boson ($gg \rightarrow H^0 $), which decays to a heavy electrically-charged Higgs bosons ($H^{\pm}$) and a $W$ boson. The $H^{\pm}$ boson subsequently decays to a second $W$ boson and the light Higgs boson, $h^0$. The light Higgs boson decays predominantly to a pair of bottom quarks, giving the process.

## Bibliotecas

In [ ]:
import shap
import xgboost
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from time import time
from matplotlib import pyplot as plt
import sys
import mplhep as hep
from scipy.special import expit 

- Modelo generado usando [XGboost](https://xgboost.readthedocs.io/en/stable/python/python_intro.html)
- De la optimización de hiperparámetros se obtiene:
 - {'eta': 0.005, 'objective': 'binary:logistic', 'subsample': 0.8, 'base_score': 0.5296727393261005, 'eval_metric': 'auc', 'max_depth': 2, 'tree_method': 'gpu_hist', 'min_child_weight': 5, 'colsample_bytree': 0.8}



## Cargando un modelo

* Descargar el modelo desde [acá](https://drive.google.com/drive/folders/1RP9mYlGoEXCaR0XemMH5LwWue8_buPpF)

In [ ]:
seed_=420
model = xgboost.Booster()
model.load_model("data/2021-09-09-binlog_auc_model.json")

## Leyendo datos

In [ ]:

df = pd.read_csv("data/higgs_bb.csv")
df.rename(columns = {'class': 'label'}, inplace = True)


df.drop(df.tail(1).index,inplace=True) # drop last n rows
df = df.apply(pd.to_numeric)


In [ ]:
# Escalando datos
y = df["label"]
X = df.iloc[:,1:]

scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)
df_scaled = pd.DataFrame(scaled_data, columns=X.columns)

## Entreaniento, validación y prueba

In [ ]:
# Training, validation, and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=seed_)

# XGBoost format
d_train = xgboost.DMatrix(X_train, label=y_train)
d_test = xgboost.DMatrix(X_test,label=y_test)

## Predicción

In [ ]:
y_pred_test = model.predict(d_test)
y_pred_train = model.predict(d_train)

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

sel_th = 0.3
fpr, tpr, ths = roc_curve(y_test,  y_pred_test)
auc_ = auc(fpr, tpr)
f1 = f1_score(y_test,  (y_pred_test>sel_th))
prec = precision_score(y_test,  (y_pred_test>sel_th))
rec = recall_score(y_test,  (y_pred_test>sel_th))
acc = accuracy_score(y_test,  (y_pred_test>sel_th))
print("F1: %.2f" %f1 , " -- prec: %.2f" %prec, " -- recall: %.2f" %rec, " -- acc: %.2f" %acc)

In [ ]:
fpr_train, tpr_train, ths_train = roc_curve(y_train, y_pred_train)
auc_train = auc(fpr_train, tpr_train)

In [ ]:
#create ROC curve
plt.figure()
plt.plot(fpr,tpr, label='ROC curve (area = %.2f) - Test' %auc_ )
plt.plot(fpr_train,tpr_train, label='ROC curve (area = %.2f) - Train' %auc_train)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Score distribution
f, axs = plt.subplots(1, 1, sharex=True, sharey=True)
h_signal_train, bins_sig_train = np.histogram(y_pred_train[y_train == 1], bins=30)
h_back_train, bins_back_train = np.histogram(y_pred_train[y_train == 0], bins=30)
h_sig_test, bins_sig_test = np.histogram(y_pred_test[y_test == 1], bins=30)
h_back_test, bins_back_test = np.histogram(y_pred_test[y_test == 0], bins=30)

axs.set_title("XGBoost")
hep.histplot([h_signal_train,h_back_train, h_sig_test, h_back_test], bins_sig_test, 
             ax=axs,label=["Train-Sig", "Train-B", "Test-S", "Test-B"])
axs.legend()
axs.set_xlabel("Score")
axs.set_ylabel("Number of Events")

plt.tight_layout()
plt.show()

## TreeExplainer

In [ ]:
import shap
# SHAP
start_time = time()
explainer = shap.TreeExplainer(model = model)
elapsed_time = time() - start_time
print("Elapsed time: %0.5f seconds." % elapsed_time)

In [ ]:
start_time = time()
shap_values = explainer.shap_values(X_test)
elapsed_time = time() - start_time
print("Elapsed time: %0.5f seconds." % elapsed_time)

## SHAP Plots

In [ ]:
# print the JS visualization code to the notebook
shap.initjs()

In [ ]:
def imprime_datos(i):
    print("Evento {} :".format(i))
    print("Pimeras 5 variables del evento:")
    display(X_test.iloc[i][0:5]) #5 primeras variables del evento i
    print("--- Etiqueta del evento {}: {}".format(i,y_test.values[i]))
    print("--- Predicción-DNN del evento {}: {}".format(i,y_pred_test[i]))
    print("--- Valor esperado del explicador: %.4f" % explainer.expected_value)
    print("--- Predicción DDN según")
    print("... 'shap_values[i].sum() + explainer.expected_value:")
    print("\t\t %.4f" % (shap_values[i].sum() + explainer.expected_value))
    print("--- Predicción DNN según")
    print("... 'logit(expit(shap_values[i].sum() + explainer.expected_value'")
    print(" \t\t %.2f" % (expit(shap_values[i].sum() + explainer.expected_value)))
    print("--- y_pred_test.mean(): %.4f" % y_pred_test.mean())
    

## Watefall plot
***
* El [shap.plots.waterfall](https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/waterfall.html) muestra la explicación de la predicción de una instancia específica.
* La parte inferior de un gráfico de cascada comienza con el valor esperado del resultado del modelo, y luego cada fila muestra cómo la contribución positiva (roja) o negativa (azul) de cada variable que define a la instancia mueve el valor del resultado esperado del modelo

In [ ]:
#https://github.com/slundberg/shap/issues/1420
i = 4
shap.plots._waterfall.waterfall_legacy(explainer.expected_value, 
                                       shap_values[i],
                                       feature_names = df.columns[1:], 
                                       show = True,
                                       max_display=12)

In [ ]:
imprime_datos(i)

In [ ]:
print(X_test.columns)

In [ ]:
print("SHAP Value m_bb: %.2f" %shap_values[i,-3])
print("SHAP Value m_wwbb: %.2f" %shap_values[i,-1])
print("SHAP Value missing energy phi: %.2f" %shap_values[i,4])

## Force plot
***
* El [gráfico de fuerza](https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/tree_based_models/Force%20Plot%20Colors.html) es otra forma de visualizar la contribución de cada variable en la predicción generada por el modelo
* Notar que se muestran los valores de las variables


In [ ]:
sh = shap.force_plot(explainer.expected_value, shap_values[i,:], X_test.iloc[i,:], link="logit")
display(sh)

## Summary Plot
***
* Muestra un resumen de como las variables de un conjunto de datos afecta en salida del modelo. 
* Cada punto del gráfico es una instancia 
* La posición del punto en el eje $x$ está determimada por su el valor SHAP 
* Los puntos se van "apilando" a lo largo de cada fila de variables 
* El color se usa para mostrar el valor original de una característica 
* Por ejemplo, en este gráfico se ve que la variable que más contribuye en promedio es m_bb.


In [ ]:
X_test.columns

In [ ]:
shap.summary_plot(shap_values, X_test)

## Ranking de variables
***
* El summary plot también se puede mostrar con barras, lo que entrega un *ranking* de variables
* Este ranking nos entrega una **explicación global** y se construye calculando un índice de importance $I_j$ de la variable $j$, considerando el promedio de los valores absolutos de los valores SHAP por cada variable y para todo el conjunto de datos:
$$I_j = \frac{1}{n}\sum_{i=1}^{n}|\phi_{j}^{(i)}|$$

donde $j$ es el índice asociado a la variable y $n$ representa el número de variables que definen a los datos

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar")

## Dependence plot
***
* Este gráfico de [dependencia](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.dependence_plot.html), también nos entrega una explicación global.
* Este gráfico muestra el valor de la variable en el eje $x$ y el valor SHAP en el eje $y$
* Se visualiza cómo el modelo depende se comporta para la variable indicada 
* La dispersión vertical de los puntos de datos representa efectos de interacción. 
* El color de los puntos va asociado a la variable con la que tiene mayor *interacción*.

In [ ]:
shap.dependence_plot("m_bb", shap_values, X_test, interaction_index=None)